In [1]:
# !pip install matplotlib
# !pip install seaborn

In [2]:
# !pip install numpy==1.23
import warnings
warnings.filterwarnings('ignore')
import os
import random
import json
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
%matplotlib inline

In [3]:
from scipy import stats
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import folium
from folium import plugins
from folium.plugins import HeatMap

In [4]:
with open('nash_contents.pkl','rb') as f:
    nash_contents = pickle.load(f)

In [5]:
nash_contents.head()

,business_id,name,address,postal_code,latitude,longitude,stars_x,review_count,is_open,attributes,categories,hours,sep_categories,Dominant Ids,Percent,Topics,Features,stars_y
0,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,37207,36.208102,-86.768170,1.5,10,1,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '...","[Ice Cream & Frozen Yogurt, Fast Food, Burgers...","[2, 3, 0]","[0.49772567, 0.21415414, 0.08519216]","[Breakfast and Coffee, Fried Chicken and Fast ...","[Ice Cream & Frozen Yogurt, Fast Food, Burgers...",2.166667
1,4iRzR7OaS-QaSXuvYxEGKA,Super Dog,1160 Gallatin Pike S,37115,36.248570,-86.719986,4.0,6,0,"{'RestaurantsReservations': 'False', 'Restaura...","Hot Dogs, Restaurants",None,"[Hot Dogs, Restaurants]","[0, 4, 1]","[0.33991277, 0.24837816, 0.18150684]","[Pizza and Italian Food, Casual Dining and Bar...","[Hot Dogs, Restaurants, Pizza and Italian Food...",3.800000
2,tMkwHmWFUEXrC9ZduonpTg,The Green Pheasant,215 1st Ave S,37201,36.159886,-86.773197,4.0,161,0,"{'RestaurantsGoodForGroups': 'True', 'HappyHou...","Restaurants, Japanese, Seafood","{'Wednesday': '16:0-22:0', 'Thursday': '16:0-2...","[Restaurants, Japanese, Seafood]","[0, 4, 1]","[0.32599193, 0.31109053, 0.14172961]","[Pizza and Italian Food, Casual Dining and Bar...","[Restaurants, Japanese, Seafood, Pizza and Ita...",4.153153
3,lk9IwjZXqUMqqOhM774DtQ,Caviar & Bananas,2031 Broadway,37203,36.148371,-86.798895,3.5,159,0,"{'RestaurantsTakeOut': 'True', 'RestaurantsRes...","Coffee & Tea, Restaurants, Wine Bars, Bars, Ni...","{'Monday': '7:0-17:0', 'Tuesday': '7:0-17:0', ...","[Coffee & Tea, Restaurants, Wine Bars, Bars, N...","[1, 3, 0]","[0.74089307, 0.083182305, 0.055624004]","[Mexican Food, Fried Chicken and Fast Food, Pi...","[Coffee & Tea, Restaurants, Wine Bars, Bars, N...",3.547009
4,oaboaRBUgGjbo2kfUIKDLQ,Mike's Ice Cream,129 2nd Ave N,37201,36.162649,-86.775973,4.5,593,1,"{'RestaurantsGoodForGroups': 'True', 'GoodForK...","Ice Cream & Frozen Yogurt, Coffee & Tea, Resta...","{'Monday': '8:0-23:0', 'Tuesday': '8:0-23:0', ...","[Ice Cream & Frozen Yogurt, Coffee & Tea, Rest...","[4, 3, 2]","[0.77526134, 0.07027646, 0.041649416]","[Casual Dining and Bars, Fried Chicken and Fas...","[Ice Cream & Frozen Yogurt, Coffee & Tea, Rest...",4.390741


In [6]:
# Reading the json file
user_ids = {}

# Loop through each file from 1 to 8
for i in range(1, 9):
    filename = f"rec_test_{i}.json"
    try:
        with open(filename, 'r') as fp:
            # Load data from the file
            data = json.load(fp)
            # Merge the loaded data into the user_ids dictionary
            user_ids.update(data)
    except FileNotFoundError:
        # If file not found, continue to the next iteration
        continue
zips=pd.read_csv('nash_zip.csv',sep=',', quotechar='"', converters={'geopoint': eval})

In [7]:
zip_codes = nash_contents['postal_code'].unique().tolist()

In [8]:
zips['postal code']=zips['postal code'].astype(str)

In [9]:
zip_site = list(zips['postal code'])

In [10]:
[item for item in zip_codes if item not in zip_site]

['']

In [11]:
locations = {str(item['postal code']):item['geopoint'] for index, item in zips.iterrows() if str(item['postal code']) in zip_codes}

In [12]:
[(item,item) for item in list(map(str,sorted([int(item) for item in sorted(locations.keys())])))]

[('28801', '28801'),
 ('32709', '32709'),
 ('37011', '37011'),
 ('37013', '37013'),
 ('37024', '37024'),
 ('37026', '37026'),
 ('37027', '37027'),
 ('37067', '37067'),
 ('37072', '37072'),
 ('37076', '37076'),
 ('37115', '37115'),
 ('37135', '37135'),
 ('37138', '37138'),
 ('37189', '37189'),
 ('37201', '37201'),
 ('37203', '37203'),
 ('37204', '37204'),
 ('37205', '37205'),
 ('37206', '37206'),
 ('37207', '37207'),
 ('37208', '37208'),
 ('37209', '37209'),
 ('37210', '37210'),
 ('37211', '37211'),
 ('37212', '37212'),
 ('37213', '37213'),
 ('37214', '37214'),
 ('37215', '37215'),
 ('37216', '37216'),
 ('37217', '37217'),
 ('37218', '37218'),
 ('37219', '37219'),
 ('37220', '37220'),
 ('37221', '37221'),
 ('37228', '37228'),
 ('37229', '37229'),
 ('37232', '37232'),
 ('37238', '37238'),
 ('37243', '37243'),
 ('37246', '37246')]

In [13]:
locations.keys()

dict_keys(['32709', '37203', '37189', '37204', '37210', '37232', '37026', '37205', '37220', '37214', '37215', '37216', '37221', '37011', '37207', '37213', '37229', '37067', '37201', '37217', '37219', '28801', '37115', '37208', '37072', '37212', '37228', '37246', '37013', '37076', '37211', '37135', '37209', '37218', '37243', '37238', '37024', '37027', '37138', '37206'])

In [14]:
topic_mapping = {
    0: 'Pizza and Italian Food',
    1: 'Mexican Food',
    2: 'Breakfast and Coffee',
    3: 'Fried Chicken and Fast Food',
    4: 'Casual Dining and Bars',
    5: 'Barbecue'
}

def map_topic(num):
    return topic_mapping[num]

In [15]:
nash_contents['Topics'] = nash_contents['Dominant Ids'].apply(lambda x: list(map(map_topic,x)))

In [16]:
# approximate radius of earth in km
def distance(coord1,coord2):
    R = 6373.0
    lat1,lon1 = coord1
    lat2,lon2 = coord2
    lat1,lon1 = np.radians(lat1), np.radians(lon1)
    lat2,lon2 = np.radians(lat2), np.radians(lon2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    c = 2*np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    dist = R*c/1.6
    return dist

coord1 = (36.2296756, -115.0122287)
coord2 = (36.2006374, -115.2251681)
print(f"The distance between the two is about {int(distance(coord1,coord2)):2} miles.")

The distance between the two is about 12 miles.


In [17]:
zip_std = '37210'
for zip in ['32709', '37203', '37189', '37204']:
    if (distance(locations[zip_std],locations[zip])) < 20.0:
        print(f'The distance from {zip_std} to {zip} {locations[zip]} is {(distance(locations[zip_std],locations[zip])):9.3f} miles')

The distance from 37210 to 37203 [36.1504, -86.7916] is     2.971 miles
The distance from 37210 to 37189 [36.2744, -86.8292] is    10.702 miles
The distance from 37210 to 37204 [36.1067, -86.7743] is     2.864 miles


In [18]:
nash_contents['Features'] = nash_contents['Features'].apply(lambda x: [item for item in x if item not in ['Restaurants','Food']])

In [19]:
nash_contents['Combined Features'] = (nash_contents['Features']).apply(lambda x: ' '.join(x))

In [24]:
nash_contents.head()

,business_id,name,address,postal_code,latitude,longitude,stars_x,review_count,is_open,attributes,categories,hours,sep_categories,Dominant Ids,Percent,Topics,Features,stars_y,Combined Features
0,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,37207,36.208102,-86.768170,1.5,10,1,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '...","[Ice Cream & Frozen Yogurt, Fast Food, Burgers...","[2, 3, 0]","[0.49772567, 0.21415414, 0.08519216]","[Breakfast and Coffee, Fried Chicken and Fast ...","[Ice Cream & Frozen Yogurt, Fast Food, Burgers...",2.166667,Ice Cream & Frozen Yogurt Fast Food Burgers Br...
1,4iRzR7OaS-QaSXuvYxEGKA,Super Dog,1160 Gallatin Pike S,37115,36.248570,-86.719986,4.0,6,0,"{'RestaurantsReservations': 'False', 'Restaura...","Hot Dogs, Restaurants",None,"[Hot Dogs, Restaurants]","[0, 4, 1]","[0.33991277, 0.24837816, 0.18150684]","[Pizza and Italian Food, Casual Dining and Bar...","[Hot Dogs, Pizza and Italian Food, Casual Dini...",3.800000,Hot Dogs Pizza and Italian Food Casual Dining ...
2,tMkwHmWFUEXrC9ZduonpTg,The Green Pheasant,215 1st Ave S,37201,36.159886,-86.773197,4.0,161,0,"{'RestaurantsGoodForGroups': 'True', 'HappyHou...","Restaurants, Japanese, Seafood","{'Wednesday': '16:0-22:0', 'Thursday': '16:0-2...","[Restaurants, Japanese, Seafood]","[0, 4, 1]","[0.32599193, 0.31109053, 0.14172961]","[Pizza and Italian Food, Casual Dining and Bar...","[Japanese, Seafood, Pizza and Italian Food, Ca...",4.153153,Japanese Seafood Pizza and Italian Food Casual...
3,lk9IwjZXqUMqqOhM774DtQ,Caviar & Bananas,2031 Broadway,37203,36.148371,-86.798895,3.5,159,0,"{'RestaurantsTakeOut': 'True', 'RestaurantsRes...","Coffee & Tea, Restaurants, Wine Bars, Bars, Ni...","{'Monday': '7:0-17:0', 'Tuesday': '7:0-17:0', ...","[Coffee & Tea, Restaurants, Wine Bars, Bars, N...","[1, 3, 0]","[0.74089307, 0.083182305, 0.055624004]","[Mexican Food, Fried Chicken and Fast Food, Pi...","[Coffee & Tea, Wine Bars, Bars, Nightlife, Ame...",3.547009,Coffee & Tea Wine Bars Bars Nightlife American...
4,oaboaRBUgGjbo2kfUIKDLQ,Mike's Ice Cream,129 2nd Ave N,37201,36.162649,-86.775973,4.5,593,1,"{'RestaurantsGoodForGroups': 'True', 'GoodForK...","Ice Cream & Frozen Yogurt, Coffee & Tea, Resta...","{'Monday': '8:0-23:0', 'Tuesday': '8:0-23:0', ...","[Ice Cream & Frozen Yogurt, Coffee & Tea, Rest...","[4, 3, 2]","[0.77526134, 0.07027646, 0.041649416]","[Casual Dining and Bars, Fried Chicken and Fas...","[Ice Cream & Frozen Yogurt, Coffee & Tea, Sand...",4.390741,Ice Cream & Frozen Yogurt Coffee & Tea Sandwic...


In [25]:
with open('nash_restaurants.pkl', 'wb') as fp:
    pickle.dump(nash_contents.drop(columns=['stars_x']).rename(columns={'stars_y':'stars'}), fp)

In [74]:
nash_contents.head()

,business_id,name,address,postal_code,latitude,longitude,stars_x,review_count,is_open,attributes,categories,hours,sep_categories,Dominant Ids,Percent,Topics,Features,stars_y,Combined Features
0,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,37207,36.208102,-86.768170,1.5,10,1,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '...","[Ice Cream & Frozen Yogurt, Fast Food, Burgers...","[2, 3, 0]","[0.49772567, 0.21415414, 0.08519216]","[Breakfast and Coffee, Fried Chicken and Fast ...","[Ice Cream & Frozen Yogurt, Fast Food, Burgers...",2.166667,Ice Cream & Frozen Yogurt Fast Food Burgers Br...
1,4iRzR7OaS-QaSXuvYxEGKA,Super Dog,1160 Gallatin Pike S,37115,36.248570,-86.719986,4.0,6,0,"{'RestaurantsReservations': 'False', 'Restaura...","Hot Dogs, Restaurants",None,"[Hot Dogs, Restaurants]","[0, 4, 1]","[0.33991277, 0.24837816, 0.18150684]","[Pizza and Italian Food, Casual Dining and Bar...","[Hot Dogs, Pizza and Italian Food, Casual Dini...",3.800000,Hot Dogs Pizza and Italian Food Casual Dining ...
2,tMkwHmWFUEXrC9ZduonpTg,The Green Pheasant,215 1st Ave S,37201,36.159886,-86.773197,4.0,161,0,"{'RestaurantsGoodForGroups': 'True', 'HappyHou...","Restaurants, Japanese, Seafood","{'Wednesday': '16:0-22:0', 'Thursday': '16:0-2...","[Restaurants, Japanese, Seafood]","[0, 4, 1]","[0.32599193, 0.31109053, 0.14172961]","[Pizza and Italian Food, Casual Dining and Bar...","[Japanese, Seafood, Pizza and Italian Food, Ca...",4.153153,Japanese Seafood Pizza and Italian Food Casual...
3,lk9IwjZXqUMqqOhM774DtQ,Caviar & Bananas,2031 Broadway,37203,36.148371,-86.798895,3.5,159,0,"{'RestaurantsTakeOut': 'True', 'RestaurantsRes...","Coffee & Tea, Restaurants, Wine Bars, Bars, Ni...","{'Monday': '7:0-17:0', 'Tuesday': '7:0-17:0', ...","[Coffee & Tea, Restaurants, Wine Bars, Bars, N...","[1, 3, 0]","[0.74089307, 0.083182305, 0.055624004]","[Mexican Food, Fried Chicken and Fast Food, Pi...","[Coffee & Tea, Wine Bars, Bars, Nightlife, Ame...",3.547009,Coffee & Tea Wine Bars Bars Nightlife American...
4,oaboaRBUgGjbo2kfUIKDLQ,Mike's Ice Cream,129 2nd Ave N,37201,36.162649,-86.775973,4.5,593,1,"{'RestaurantsGoodForGroups': 'True', 'GoodForK...","Ice Cream & Frozen Yogurt, Coffee & Tea, Resta...","{'Monday': '8:0-23:0', 'Tuesday': '8:0-23:0', ...","[Ice Cream & Frozen Yogurt, Coffee & Tea, Rest...","[4, 3, 2]","[0.77526134, 0.07027646, 0.041649416]","[Casual Dining and Bars, Fried Chicken and Fas...","[Ice Cream & Frozen Yogurt, Coffee & Tea, Sand...",4.390741,Ice Cream & Frozen Yogurt Coffee & Tea Sandwic...


In [27]:
topic = 'Pizza and Italian Food'
get_restaurants_by_topic(nash_contents,categories_list=[topic])[:2]


[{'Name': 'Two Goats',
  'Address': '4611 Alabama Ave',
  'Zip Code': '37206',
  'Latitude': 36.1530043,
  'Longitude': -86.843325,
  'Category': ['Personal Chefs',
   'Food',
   'American (New)',
   'Caterers',
   'Event Planning & Services',
   'Pop-Up Restaurants',
   'Restaurants',
   'Food Trucks'],
  'Topics': ['Breakfast and Coffee',
   'Pizza and Italian Food',
   'Casual Dining and Bars'],
  'Stars': 5.0},
 {'Name': "Yogi's Pizzeria",
  'Address': '4825 Trousdale Dr',
  'Zip Code': '37220',
  'Latitude': 36.075488,
  'Longitude': -86.760279,
  'Category': ['Pizza', 'Ice Cream & Frozen Yogurt', 'Food', 'Restaurants'],
  'Topics': ['Pizza and Italian Food',
   'Casual Dining and Bars',
   'Mexican Food'],
  'Stars': 5.0}]

In [28]:
preference = 'Chicken Shop'
get_restaurants_by_preference(nash_contents,categories_list=[preference])[:2]

[{'Name': "Riddim N' Spice",
  'Address': '2116 Meharry Blvd',
  'Zip Code': '37208',
  'Latitude': 36.1678891,
  'Longitude': -86.8106775,
  'Category': ['Restaurants',
   'Chicken Shop',
   'Caribbean',
   'Vegan',
   'Food Trucks',
   'Vegetarian',
   'Food'],
  'Topics': ['Fried Chicken and Fast Food',
   'Breakfast and Coffee',
   'Barbecue'],
  'Stars': 4.716981132075472},
 {'Name': 'Coneheads',
  'Address': '1315 Dickerson Pike',
  'Zip Code': '37207',
  'Latitude': 36.1954555,
  'Longitude': -86.7717418,
  'Category': ['Chicken Shop', 'Restaurants'],
  'Topics': ['Barbecue', 'Breakfast and Coffee', 'Mexican Food'],
  'Stars': 4.69811320754717}]

In [29]:
preference1 = ['Chinese']
preference2 = ['Indian']
topic = ['Breakfast and Coffee']
store_rest = []

In [30]:
rest_pref1 = get_restaurants_by_preference(nash_contents,preference1)
rest_pref2 = get_restaurants_by_preference(nash_contents,preference2)
pref_store = [rest_pref1,rest_pref2]

In [31]:
if preference1 != preference2:
    for i in range(2):
        store_rest.extend(pref_store[i][:10])
else:
    store_rest.extend(pref_store[0][:20])
store_rest.extend(get_restaurants_by_topic(nash_contents,topic)[:10])

In [32]:
zip_code = '37232'
proximity = 20.0
for item in store_rest:
    dist = distance(locations[zip_code],locations[item['Zip Code']])    
    if dist < proximity:        
        print(f'The distance from {zip_code} is {dist:9.3f} miles')

The distance from 37232 is     4.761 miles
The distance from 37232 is     1.965 miles
The distance from 37232 is     4.761 miles
The distance from 37232 is     6.838 miles
The distance from 37232 is     2.542 miles
The distance from 37232 is     1.450 miles
The distance from 37232 is     3.778 miles
The distance from 37232 is     9.063 miles
The distance from 37232 is     4.197 miles
The distance from 37232 is     5.588 miles
The distance from 37232 is     8.644 miles
The distance from 37232 is     7.161 miles
The distance from 37232 is    13.067 miles
The distance from 37232 is     8.644 miles
The distance from 37232 is     4.761 miles
The distance from 37232 is     8.644 miles
The distance from 37232 is     2.542 miles
The distance from 37232 is     2.542 miles
The distance from 37232 is     2.542 miles
The distance from 37232 is     2.542 miles
The distance from 37232 is     4.761 miles
The distance from 37232 is     4.761 miles
The distance from 37232 is     2.519 miles
The distanc

In [33]:
[item for item in store_rest if distance(locations[zip_code],locations[item['Zip Code']]) < proximity][:2]

[{'Name': 'Mama Yang and Daughter',
  'Address': '4611 Alabama Ave',
  'Zip Code': '37209',
  'Latitude': 36.1530043,
  'Longitude': -86.8433279,
  'Category': ['Taiwanese', 'Chinese', 'Restaurants'],
  'Topics': ['Casual Dining and Bars', 'Pizza and Italian Food', 'Barbecue'],
  'Stars': 4.857142857142857},
 {'Name': "Tang's Chinese Restaurant",
  'Address': '5814 Nolensville Pike, Ste 110',
  'Zip Code': '37246',
  'Latitude': 36.16578,
  'Longitude': -86.78442,
  'Category': ['Restaurants', 'Chinese'],
  'Topics': ['Pizza and Italian Food',
   'Fried Chicken and Fast Food',
   'Breakfast and Coffee'],
  'Stars': 4.666666666666667}]

In [35]:
nash_coordinates = (36.174465, - -86.767960)
folium_map = folium.Map(location=nash_coordinates,width=1000,height=800,zoom_start=11)

In [36]:
for restaurant in store_rest:
    coordinates = restaurant['Latitude'],restaurant['Longitude']
    folium.Marker(coordinates,popup=(restaurant['Name'],restaurant['Category'])).add_to(folium_map)

In [37]:
data = [(restaurant['Latitude'],restaurant['Longitude']) for restaurant in store_rest]
HeatMap(data).add_to(folium.FeatureGroup(name='Heat Map').add_to(folium_map))


In [38]:
folium_map

In [39]:
# with open('restaurants.pkl','rb') as f:
#     restaurants = pickle.load(f)

In [40]:
tfidf = TfidfVectorizer()

In [42]:
tfidf_matrix = tfidf.fit_transform(nash_contents['Combined Features'])

In [44]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [45]:
cosine_sim[0]

array([1.        , 0.17964523, 0.19388992, ..., 0.37163229, 0.37964471,
       0.17722586])

In [46]:
nash_contents.index

RangeIndex(start=0, stop=2458, step=1)

In [47]:
nash_contents['business_id']

0       bBDDEgkFA1Otx9Lfe7BZUQ
1       4iRzR7OaS-QaSXuvYxEGKA
2       tMkwHmWFUEXrC9ZduonpTg
3       lk9IwjZXqUMqqOhM774DtQ
4       oaboaRBUgGjbo2kfUIKDLQ
                 ...          
2453    5quqExjLtFNhykec_na8pg
2454    V-snSOHDO_mrWOBtJHtg4w
2455    -KNgMklrxgRX2-T6xXeTZA
2456    n6W7Dtz5Xz4glh2bkEFX1Q
2457    2MAQeAqmD8enCT2ZYqUgIQ
Name: business_id, Length: 2458, dtype: object

In [48]:
businesses = nash_contents['business_id']
indices = pd.Series(businesses.index,index = nash_contents['business_id'])

In [49]:
def content_recommendations(business):
    idx = indices[business]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    biz_indices = [i[0] for i in sim_scores]
    return businesses.iloc[biz_indices].values.tolist()

In [50]:
content_recommendations('bBDDEgkFA1Otx9Lfe7BZUQ')

['Q4tTrdDJ95_PviqNh6w74A',
 'uE0iQih1BfnQUTUxI3iFMw',
 'HlSve6o5TAXbl4UDwCfcFw',
 'zr3z36d9rRX68wKcHstOmA',
 'nT0acgBkVSEelbNTXRSFbg',
 'vwxHvJSDGYM75tmjKcMXtg',
 'G3Zd640VOpMgxXizb6aO6A',
 'quk1d6aRxJf4amQnf49F9Q',
 'ak5cTOoVvZLaRcyqg05K4w',
 'WSmLr421oStUdizBmBYPpw',
 'rYJpJq1LIt-mIVpQ1WSjjA',
 'PxVFM9j9IbjBMlMqvisqnA',
 '8F6OsZYrl3LbR17teMJWJQ',
 'iGTrw3hno7Yf6e6B98VZMA',
 'CxQ5GonUhGjgfX8REEFjdw',
 'rQ5a0LlEYyH5ld3k9V9NmQ',
 'soFyg4gZqg0YzrDCLk6Jxw',
 '-7CqBmK0zxwyjEqa3LBVSw',
 'oaboaRBUgGjbo2kfUIKDLQ',
 'GNhgVpSoCC8xAz8URfmzhQ']

In [51]:
nash_reviews = pd.read_pickle('nash_reviews.pkl')

In [53]:
nash_reviews['date'] = nash_reviews['date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))

In [54]:
nash_reviews_open = nash_reviews[nash_reviews['business_id'].isin(nash_contents['business_id'])]

In [55]:
nash_reviews_open.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
39,E9AB7V4z8xrt2uPF7T55FQ,iYY5Ii1LGpZCpXFkHlMefw,Zx7n8mdt8OzLRXVzolXNhQ,5.0,0,0,0,Amazing biscuits and (fill in the blank). Grea...,2018-04-27 23:03:21
43,qGQvUHmC02PAZW3H1WTIfw,RRTQpg8hutdimzAYuP_Hbw,eaJCpC6IhYphj7bwCDHTwQ,5.0,0,0,0,Stopped by after a Sunday morning walk in the ...,2015-08-30 13:41:47
63,67cGcRrS0jTvB8p7rpaPew,8YSxVaD203mE_9FR4nCEVA,oQ5CPRt0R3AzFvcjNOqB1w,3.0,1,0,0,Honestly the food doesn't knock my socks off b...,2016-01-17 13:26:19
82,DOeEDBfsrNph0ktNkYYFnQ,qVYilGRmY6uemAy_tDpfuA,9OtFX3YzG5keaUMOWU1imA,5.0,4,0,0,Man. This place is nice. The bartender was awe...,2014-05-26 01:56:19
91,xumAI7br1X67OY_bU_Yqhg,jEmClJb6wjfDUksVMO_Jjg,X_E7U2lVNEwbH_gpOBmAHQ,5.0,0,0,0,This place was so chill and relaxing. When I c...,2017-06-17 17:46:55


In [56]:
last_restaurants = nash_reviews_open[['user_id','business_id','date']].merge(nash_reviews_open.groupby('user_id')['date'].max(),on=['user_id','date'])

In [57]:
usr_biz = []

for usr in user_ids.keys():
    user_data = last_restaurants[last_restaurants['user_id'] == usr]['business_id']
    if not user_data.empty:
        usr_biz.append((usr, user_data.iloc[0]))

In [58]:
store_restaurants_content = {item[0]:content_recommendations(item[1]) for item in usr_biz}

In [59]:
with open('rec_content.json', 'w') as fp:
    json.dump(store_restaurants_content, fp)

In [60]:
test_user = '2XKlUjHGk9z-2ckFMcBEHw'

In [61]:
store_restaurants_content[test_user]

['Icu-z8o386uHuj0bB345Cg',
 'MJgHI9yuGej00bemw0R_uw',
 'JWUkEIayb3F8la3TWe-eyw',
 'LNMi7I89I9NI3bQz8LPetA',
 'nD-cMgRg5gekeB9qbuEkeA',
 'JgSOSivikoK_jn5l8KJ9Sw',
 '8Wi6ExrCwlUtnhXk7at95A',
 '3XoqdiwUGX81F3rsRNkNbA',
 '7WaVMXsqN2nNc3vTNCOOjw',
 'XrikSy0DQ_fhVCaRRBJI-g',
 'ijiuTK4UjT0wGBGP6fQq5Q',
 '1165tVWCcxFvYoHT1-I2mw',
 '4fUrcm8sFSprCbHqa8pdog',
 'wa9tG3QDiR5Qx8P_5oIwdQ',
 'c6fWBSm-EHVwyFszTC5Mhw',
 'xXgE_rj-tK9v_MIH3mXpaQ',
 'u41uezgA8pp1wrNrpP1HvQ',
 'MPCknINXdfg1aWnBK7B7gQ',
 'W-TAJwlfKb2DQ68m-fwqqw',
 'vC2Yt0Cdm5x0K-a78vQU8g']

In [62]:
def get_restaurants_by_usr_id(df,usr_id):
    
    store_restaurants = []
    
    df_colab = nash_contents[nash_contents['business_id'].isin([item[0] for item in user_ids[usr_id]])]
    df_content = nash_contents[nash_contents['business_id'].isin([item for item in store_restaurants_content[usr_id]])]
    df_rest = pd.concat([df_colab,df_content])
     
    for index,row in df_rest.iterrows():
        restaurant_details = {}
        restaurant_details['Name'] = row['name']
        restaurant_details['Address'] = row['address']
        restaurant_details['Zip Code'] = row['postal_code']
        restaurant_details['Latitude'] = row['latitude']
        restaurant_details['Longitude'] = row['longitude']
        
        restaurant_details['Category'] = row['sep_categories']
        restaurant_details['Topics'] = row['Topics']
        restaurant_details['Stars'] = row['stars_y']
#         restaurant_details['additional_details'] = row['attributes']
        store_restaurants.append(restaurant_details)
    return sorted(store_restaurants,key = lambda x: x['Stars'],reverse=True)

In [63]:
store = get_restaurants_by_usr_id(nash_contents,test_user)
[item for item in store if distance(locations[zip_code],locations[item['Zip Code']]) < proximity]

[{'Name': 'Brightside Bakeshop',
  'Address': '713 Porter Rd',
  'Zip Code': '37206',
  'Latitude': 36.1828547,
  'Longitude': -86.7325393,
  'Category': ['Bakeries',
   'Coffee & Tea',
   'Breakfast & Brunch',
   'Restaurants',
   'Food'],
  'Topics': ['Fried Chicken and Fast Food',
   'Breakfast and Coffee',
   'Barbecue'],
  'Stars': 5.0},
 {'Name': 'Peachtree Grill',
  'Address': '329 Peachtree St',
  'Zip Code': '37210',
  'Latitude': 36.1202411,
  'Longitude': -86.7490427,
  'Category': ['American (New)',
   'Mediterranean',
   'Middle Eastern',
   'Burgers',
   'American (Traditional)',
   'Restaurants'],
  'Topics': ['Mexican Food',
   'Fried Chicken and Fast Food',
   'Breakfast and Coffee'],
  'Stars': 4.837209302325581},
 {'Name': 'Lazzaroli Pasta',
  'Address': '1314 5th Ave N',
  'Zip Code': '37208',
  'Latitude': 36.1778436,
  'Longitude': -86.7890357,
  'Category': ['Restaurants',
   'Food',
   'Pasta Shops',
   'Ethnic Food',
   'Italian',
   'Specialty Food',
   'Groce

### Cold Start

In [65]:
# Calculate the mean star rating for each restaurant
restaurant_means = nash_contents.groupby('business_id')['stars_y'].mean().reset_index()

# Merge mean star ratings back to the original DataFrame
nash_contents_with_means = pd.merge(nash_contents, restaurant_means, on='business_id', suffixes=('', '_mean'))

# Group restaurants by postal code and find the top 10 highest rated restaurants in each group
top_10_by_postal_code = nash_contents_with_means.groupby('postal_code').apply(lambda x: x.nlargest(10, 'stars_y_mean'))

# Reset index to flatten the DataFrame
top_10_by_postal_code = top_10_by_postal_code.reset_index(drop=True)

In [73]:
top_10_by_postal_code.loc[top_10_by_postal_code['postal_code']!=''][['name','address','postal_code','latitude','longitude','sep_categories','Topics','stars_y']].rename(columns={'stars_y':'stars'}).to_csv('cold_start.csv',index=False)